In [6]:
import pandas as pd


def importar():
    """importa os dados necessários e devolve dois dados: tabela da malária e dados dos municípios
    arg: nenhum
    retunr: df =  tabela pandas para base de dados e municipios = tabela pandas dos municipios e da população por ano."""
    df = pd.read_pickle("dados\Dados_utilizaveis")
    # criar coluna com dados de casos confirmados como 1, outros resultados como 0
    df["detection.numerico"] = df["detection.type"].map(
        {"active": 1, "passive": 0, "CVL": 0}
    )
    municipio = pd.read_pickle("dados\municipio")
    municipio["id_municipio"] = (
        municipio["id_municipio"] // 10
    )  # Tirado ultimo digito do ID para facilitar seu uso, pois não possue diferencia retirar esse digito para o uso final.
    municipios_localizacao = pd.read_csv(
        "dados\municipios_latitude_longitude.csv"
    )  # importar dados de latitudes e longitudes por município
    municipios_localizacao["codigo_ibge"] = (
        municipios_localizacao["codigo_ibge"] // 10
    )  # Tirado ultimo digito do ID para facilitar seu uso, pois não possue diferencia retirar esse digito para o uso final.
    return df, municipio, municipios_localizacao


In [7]:
# criar coluna com ocupação
def populacao(dados, municipio_dados):
    """Devolve uma lista de dados com a população de cada linha, baseado nos municipios e ano
    Arg: dados de casos e dados dos municipios
    return: lista de população"""
    lista_populacao = []
    ano_anterior = 1
    municipio_code_anterio = 1
    for i in range(len(dados)):
        ano_novo = int(dados.loc[i, "data"].year)
        municipio_code_novo = int(dados.loc[i, "infection.county"])
        if (ano_anterior != ano_novo) or (
            municipio_code_anterio != municipio_code_novo
        ):
            logic1 = municipio_dados["ano"] == ano_novo
            logic2 = municipio_dados["id_municipio"] == municipio_code_novo
            populacao_numero = int(municipio_dados.loc[logic1 & logic2]["populacao"])
            ano_anterior = ano_novo
            municipio_code_anterio = municipio_code_novo
        lista_populacao.append(populacao_numero)
    return lista_populacao


In [8]:
def localizacao(dados, municipio_localizacao):
    """Devolve duas lista de dados com a latitude e longitude respectivamente de cada linha, baseado nos municipios
    Arg: dados de casos e dados dos municipios
    return: lista de latitude e lista de longitude"""
    lista_latitude = []
    lista_longitude = []
    municipio_code_anterio = 1
    for i in range(len(dados)):
        municipio_code_novo = int(dados.loc[i, "infection.county"])
        if municipio_code_anterio != municipio_code_novo:
            logic1 = municipio_localizacao["codigo_ibge"] == municipio_code_novo
            latitude_numero = float(municipio_localizacao.loc[logic1]["latitude"])
            longitude_numero = float(municipio_localizacao.loc[logic1]["longitude"])
            municipio_code_anterio = municipio_code_novo
        lista_latitude.append(latitude_numero)
        lista_longitude.append(longitude_numero)
    return lista_latitude, lista_longitude


In [9]:
from sklearn.preprocessing import OneHotEncoder
from seaborn import load_dataset


def onehot_encoder(dados, coluna):
    """Faz o onehot para a coluna espeficidade dentro de um dataframe.
    arg: dados e nome da coluna
    return matrix numpy do onehot da coluna"""
    df = dados[[coluna]]
    ohe = OneHotEncoder()
    transformed = ohe.fit_transform(df)
    merge = transformed.toarray()
    return merge

In [12]:
import numpy as np

df, municipio_dados, municipio_localizacao = importar()
# Apagar dados de 2009
logic2009 = df["notification.year"] != 2009
df = df.loc[logic2009]
# renomear colunas
df.rename(
    columns={"notification.year": "year", "notification.month": "month"}, inplace=True
)
# criar coluna de dias
df["day"] = np.ones(len(df))
# criar coluna de datas e deletar colunas de mes, ano e dias
df["data"] = pd.to_datetime(df[["year", "month", "day"]])
df = df.drop(columns=["month", "year", "day"])
# criar dados trimestrais
dados_finais = df.groupby(
    [pd.Grouper(key="data", freq="Q"), "infection.county", "occupation"]
).sum()
# separar dados por mes
# dados_finais = df.groupby(["data","infection.county", "occupation"]).sum()
# apagar dados com somatoria da detecção igual a 0
logic1 = dados_finais["detection.numerico"] > 0
dados_finais = dados_finais.loc[logic1].reset_index()
# Descobrir população de cada municipio por ano
dados_finais["populacao"] = populacao(dados_finais, municipio_dados)
dados_finais["probabilidade"] = (
    dados_finais["detection.numerico"] / dados_finais["populacao"] * 100
)
latitude, longitude = localizacao(dados_finais, municipio_localizacao)
dados_finais["latitude"] = latitude
dados_finais["longitude"] = longitude
# Onehot Encoder para a coluna occupation
matrix_onehot = onehot_encoder(dados_finais, "occupation")
dados_finais = pd.concat(
    [
        dados_finais,
        pd.DataFrame(
            matrix_onehot, columns=sorted(list(dados_finais["occupation"].unique()))
        ),
    ],
    axis=1,
)

In [13]:
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.05

indices = dados_finais.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=41
)

df_treino = dados_finais.loc[indices_treino]
df_teste = dados_finais.loc[indices_teste]

print(df_treino)
print()
print(df_teste)


            data  infection.county         occupation  detection.numerico  \
4823  2011-09-30          110080.0            grazing                  17   
13342 2015-03-31          120042.0              other                 151   
4906  2011-09-30          130002.0  road construction                   2   
13409 2015-03-31          130140.0        agriculture                  33   
19880 2018-06-30          140045.0    domestic worker                 123   
...          ...               ...                ...                 ...   
8513  2012-12-31          110110.0            panning                   4   
21584 2019-03-31          140017.0             mining                   1   
20450 2018-09-30          140020.0        agriculture                  69   
931   2010-06-30          120020.0    domestic worker                  83   
1984  2010-09-30          130270.0        agriculture                  24   

       populacao  probabilidade  latitude  longitude  agriculture  \
4823  